In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df= pd.read_csv('movie.csv')

In [3]:
df.shape

(8992, 3)

In [4]:
df.head()

,userId,movie,rating
0,3,Toy Story (1995),4.0
1,6,Toy Story (1995),5.0
2,8,Toy Story (1995),4.0
3,10,Toy Story (1995),4.0
4,11,Toy Story (1995),4.5


In [6]:
len(df.userId.unique())

4081

In [8]:
len(df.movie.unique())

10

In [9]:
len(df.rating.unique())

10

In [10]:
df.rating.unique()

array([4. , 5. , 4.5, 3. , 1. , 3.5, 1.5, 2. , 2.5, 0.5])

In [11]:
df.movie.unique()

array(['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)',
       'Waiting to Exhale (1995)', 'Father of the Bride Part II (1995)',
       'Heat (1995)', 'Sabrina (1995)', 'Tom and Huck (1995)',
       'Sudden Death (1995)', 'GoldenEye (1995)'], dtype=object)

In [13]:
df.groupby('movie')['rating'].mean().sort_values(ascending=False)

movie
Toy Story (1995)                      3.959323
Heat (1995)                           3.836508
GoldenEye (1995)                      3.427003
Sabrina (1995)                        3.381429
Tom and Huck (1995)                   3.352564
Jumanji (1995)                        3.268398
Grumpier Old Men (1995)               3.186861
Father of the Bride Part II (1995)    3.143836
Sudden Death (1995)                   3.017327
Waiting to Exhale (1995)              3.000000
Name: rating, dtype: float64

##### Cosine Similarity

##### Item-Item

In [16]:
df1=df.pivot_table(index='movie',columns='userId',values='rating')
df1.fillna(0,axis=1,inplace=True)
df1.head()

userId,1,2,3,4,5,6,7,8,10,11,12,13,14,15,16,17,19,21,22,23,24,26,29,30,31,32,34,38,39,41,46,47,48,53,54,58,59,61,66,67,...,7054,7055,7056,7058,7059,7060,7061,7064,7065,7067,7068,7070,7073,7075,7077,7080,7081,7083,7085,7087,7088,7092,7093,7094,7097,7099,7100,7101,7102,7104,7105,7107,7108,7110,7113,7115,7116,7117,7119,7120
movie,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Father of the Bride Part II (1995),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,3.5,0.0,0.0,0.0
GoldenEye (1995),0.0,0.0,0.0,4.0,0.0,0.0,0.0,4.0,0.0,2.5,0.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,3.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,...,0.0,3.0,3.0,5.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,4.0,5.0,0.0,0.0,3.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0
Grumpier Old Men (1995),0.0,4.0,0.0,0.0,0.0,3.0,3.0,5.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
Heat (1995),0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,4.0,5.0,4.0,5.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,2.0,0.0,4.0,0.0,3.0,4.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5.0,3.0,3.0,5.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,3.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,0.0
Jumanji (1995),3.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,1.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0


In [19]:
similarity= cosine_similarity(df1)

In [21]:
similarity.shape

(10, 10)

In [23]:
similarity[1]

array([0.21599275, 1.        , 0.21819777, 0.39773907, 0.40226034,
       0.22145469, 0.1552551 , 0.11972423, 0.38348625, 0.12844824])

In [28]:
df1.index

Index(['Father of the Bride Part II (1995)', 'GoldenEye (1995)',
       'Grumpier Old Men (1995)', 'Heat (1995)', 'Jumanji (1995)',
       'Sabrina (1995)', 'Sudden Death (1995)', 'Tom and Huck (1995)',
       'Toy Story (1995)', 'Waiting to Exhale (1995)'],
      dtype='object', name='movie')

In [77]:
def recommended_movie(similar_movie):
    if similar_movie in df1.index:
        index= np.where(similar_movie==df1.index)[0][0]
        similar=sorted(list(enumerate(similarity[index])),reverse=True,key=lambda x: x[1])[1:6]
        print(f'Recommended_movie of {similar_movie}')
        print('-'*30)
        for index in similar:
            print(df1.index[index[0]])
    else:
        print('Movie is not in the list')

In [79]:
recommended_movie('Tom and Huck (1995)')

Recommended_movie of Tom and Huck (1995)
------------------------------
Jumanji (1995)
Sabrina (1995)
Father of the Bride Part II (1995)
Waiting to Exhale (1995)
Grumpier Old Men (1995)


In [41]:
np.where('Sudden Death (1995)'==df1.index)[0][0]

np.int64(6)

In [68]:
similar=sorted(list(enumerate(similarity[6])),reverse=True,key=lambda x: x[1])[1:6]
for index in similar:
    print(df1.index[index[0]])

Heat (1995)
Grumpier Old Men (1995)
Father of the Bride Part II (1995)
Sabrina (1995)
Toy Story (1995)


In [43]:
list1=[1,2,3]

In [45]:
for i in enumerate(list1):
    print(i)

(0, 1)
(1, 2)
(2, 3)


##### User-user

In [80]:
df1=df.pivot_table(index='userId',columns='movie',values='rating')
df1.fillna(0,axis=1,inplace=True)
df1.head()

movie,Father of the Bride Part II (1995),GoldenEye (1995),Grumpier Old Men (1995),Heat (1995),Jumanji (1995),Sabrina (1995),Sudden Death (1995),Tom and Huck (1995),Toy Story (1995),Waiting to Exhale (1995)
userId,,,,,,,,,,
1,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
4,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0


In [87]:
df2=pd.DataFrame(cosine_similarity(df1),index=df.userId.unique(),columns=df.userId.unique())
df2.head()

,3,6,8,10,11,12,13,14,16,19,22,23,24,31,34,39,47,53,54,58,59,66,69,80,82,84,90,91,93,96,97,99,105,107,109,114,115,116,117,124,...,6560,6580,6589,6613,6622,6624,6633,6641,6644,6669,6670,6693,6703,6755,6774,6807,6820,6835,6850,6851,6868,6879,6884,6901,6909,6911,6926,6933,6962,6967,6975,6979,6993,7030,7031,7044,7070,7080,7087,7105
3,1.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.514496,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.6,0.0,0.0,0.000000,0.363803,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.390567,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.323498,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.443607,0.0,0.000000,0.491539,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,1.0,0.000000,0.0,0.0,1.0,0.707107,0.000000,0.0,0.000000,0.000000,0.0,0.553372
6,0.0,1.0,0.0,0.0,0.0,0.390567,0.707107,0.615457,0.0,0.000000,0.437595,0.000000,0.000000,0.0,0.000000,0.0,0.419314,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.457496,0.000000,0.0,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.323498,0.0,0.000000,0.000000,0.0,0.000000,0.514496,0.0,0.0,0.0,0.0,0.000000,0.0,0.169031,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.458831,0.0,0.000000
8,0.0,0.0,1.0,0.0,0.0,0.650945,0.000000,0.492366,1.0,0.874157,0.583460,0.685994,0.789352,0.0,0.707107,0.0,0.524142,0.0,0.514496,1.0,0.544331,0.0,0.0,0.0,1.0,0.000000,0.606339,0.000000,0.707107,0.0,0.00000,1.0,0.0,1.0,0.520756,0.743294,1.0,0.0,1.0,0.0,...,0.857493,0.80,0.000000,0.323498,1.0,0.658505,0.690268,0.0,0.685994,0.685994,0.0,0.0,1.0,1.0,0.000000,0.0,0.845154,0.655386,0.00,0.0,0.980581,0.0,0.0,1.0,1.0,0.000000,0.0,0.0,0.0,0.707107,0.0,1.0,0.0,0.707107,0.000000,0.0,0.752577,0.458831,1.0,0.622543
10,0.0,0.0,0.0,1.0,0.0,0.000000,0.000000,0.615457,0.0,0.388514,0.262557,0.411597,0.000000,0.8,0.424264,0.6,0.314485,0.6,0.514496,0.0,0.653197,0.8,0.0,0.0,0.0,0.989949,0.485071,0.000000,0.424264,0.0,0.33282,0.0,0.6,0.0,0.650945,0.401379,0.0,0.0,0.0,0.8,...,0.000000,0.48,0.758947,0.754829,0.0,0.602061,0.490858,0.6,0.308697,0.308697,0.6,0.8,0.0,0.0,0.785818,0.8,0.000000,0.000000,0.96,0.8,0.156893,0.6,0.6,0.0,0.0,0.189737,0.0,0.0,0.0,0.424264,0.8,0.0,0.0,0.000000,0.989949,0.0,0.000000,0.619422,0.0,0.000000
11,1.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.514496,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.6,0.0,0.0,0.000000,0.363803,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.390567,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.323498,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.443607,0.0,0.000000,0.491539,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,1.0,0.000000,0.0,0.0,1.0,0.707107,0.000000,0.0,0.000000,0.000000,0.0,0.553372


In [99]:
df2[df2[13]>0.9][13].sort_values(ascending=False)[1:6]

3559    1.000000
5953    1.000000
2511    0.997054
3259    0.993884
2155    0.993884
Name: 13, dtype: float64

In [101]:
df[(df.userId==13) | (df.userId==5953)]

,userId,movie,rating
6,13,Toy Story (1995),4.0
2167,5953,Toy Story (1995),4.0
2571,13,Jumanji (1995),3.0
7447,13,GoldenEye (1995),3.0
